In [347]:
import requests, bs4
import datetime
from bs4 import BeautifulSoup as bs
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pprint
import re
import csv
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge #ordinary linear regression + w/ ridge regularization
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor



In [348]:
data_df= pd.read_csv("detail_page_data")
data_df.head(10)


,Unnamed: 0,terrain,skis,ability_level,rocker_type,turning_radius,core_laminates,name,price,detail_link,brand,year,gender,full_link
0,0,All-Mountain,No Bindings,Advanced-Expert,Rocker/Camber/Rocker,Medium,"Carbon, Metal, Wood",Fischer Ranger 102 FR Skis 2022,699.99,/skis/fischer-ranger-102-fr,Fischer,2022,Unisex,https://www.evo.com//skis/fischer-ranger-102-fr
1,1,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Short,Wood,Rossignol Black Ops Stargazer Skis - Women's 2022,599.95,/skis/rossignol-black-ops-stargazer-womens,Rossignol,2022,Women's,https://www.evo.com//skis/rossignol-black-ops-...
2,2,"Carving, All-Mountain",With Bindings,Beginner-Intermediate,Rocker/Camber,Short,"Composite, Wood",Head V-Shape V4 Skis ​+ PR 10 GW Bindings 2022,499.00,/ski-packages/head-v-shape-v4-skis-pr-10-gw-bi...,Head,2022,Unisex,https://www.evo.com//ski-packages/head-v-shape...
3,3,"Carving, All-Mountain",With Bindings,Intermediate-Advanced,Rocker/Camber,Short,"Graphene, Polyurethane (PU), Wood",Head V-Shape V6 Skis ​+ PR 11 GW Bindings 2022,599.00,/ski-packages/head-v-shape-v6-skis-pr-11-gw-bi...,Head,2022,Unisex,https://www.evo.com//ski-packages/head-v-shape...
4,4,"Carving, All-Mountain",With Bindings,Intermediate-Advanced,Rocker/Camber,Short,"Carbon, Wood",Head Total Joy Skis ​+ Joy 11 GW SLR Bindings ...,799.00,/ski-packages/head-total-joy-skis-11-gw-slr-bi...,Head,2022,Women's,https://www.evo.com//ski-packages/head-total-j...
5,5,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,Wood,Rossignol Black Ops Sender Skis 2022,699.95,/skis/rossignol-black-ops-sender,Rossignol,2022,Unisex,https://www.evo.com//skis/rossignol-black-ops-...
6,6,"All-Mountain, Big Mountain",No Bindings,Advanced-Expert,Rocker/Camber/Rocker,Long,"Metal, Wood",K2 Mindbender 108Ti Skis 2022,749.95,/skis/k2-mindbender-108ti,K2,2022,Unisex,https://www.evo.com//skis/k2-mindbender-108ti
7,7,"Park & Pipe, All-Mountain",No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,"Carbon, Wood",K2 Poacher Skis 2022,499.95,/skis/k2-poacher,K2,2022,Unisex,https://www.evo.com//skis/k2-poacher
8,8,"Park & Pipe, All-Mountain",No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Short,Wood,K2 Reckoner 92 Skis 2022,299.95,/skis/k2-reckoner-92,K2,2022,Unisex,https://www.evo.com//skis/k2-reckoner-92
9,9,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,"Metal, Wood",Atomic Maverick 100 TI Skis 2022,749.99,/skis/atomic-maverick-100-ti,Atomic,2022,Unisex,https://www.evo.com//skis/atomic-maverick-100-ti


In [349]:
data_df= data_df.drop(columns=['Unnamed: 0'])
data_df


,terrain,skis,ability_level,rocker_type,turning_radius,core_laminates,name,price,detail_link,brand,year,gender,full_link
0,All-Mountain,No Bindings,Advanced-Expert,Rocker/Camber/Rocker,Medium,"Carbon, Metal, Wood",Fischer Ranger 102 FR Skis 2022,699.99,/skis/fischer-ranger-102-fr,Fischer,2022,Unisex,https://www.evo.com//skis/fischer-ranger-102-fr
1,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Short,Wood,Rossignol Black Ops Stargazer Skis - Women's 2022,599.95,/skis/rossignol-black-ops-stargazer-womens,Rossignol,2022,Women's,https://www.evo.com//skis/rossignol-black-ops-...
2,"Carving, All-Mountain",With Bindings,Beginner-Intermediate,Rocker/Camber,Short,"Composite, Wood",Head V-Shape V4 Skis ​+ PR 10 GW Bindings 2022,499.00,/ski-packages/head-v-shape-v4-skis-pr-10-gw-bi...,Head,2022,Unisex,https://www.evo.com//ski-packages/head-v-shape...
3,"Carving, All-Mountain",With Bindings,Intermediate-Advanced,Rocker/Camber,Short,"Graphene, Polyurethane (PU), Wood",Head V-Shape V6 Skis ​+ PR 11 GW Bindings 2022,599.00,/ski-packages/head-v-shape-v6-skis-pr-11-gw-bi...,Head,2022,Unisex,https://www.evo.com//ski-packages/head-v-shape...
4,"Carving, All-Mountain",With Bindings,Intermediate-Advanced,Rocker/Camber,Short,"Carbon, Wood",Head Total Joy Skis ​+ Joy 11 GW SLR Bindings ...,799.00,/ski-packages/head-total-joy-skis-11-gw-slr-bi...,Head,2022,Women's,https://www.evo.com//ski-packages/head-total-j...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,Wood,Lib Tech UFO 105 Skis 2022,649.99,/skis/lib-tech-ufo-105,Lib Tech,2022,Unisex,https://www.evo.com//skis/lib-tech-ufo-105
927,"Park & Pipe, All-Mountain",No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,Wood,Lib Tech UFO 95 Skis 2022,599.99,/skis/lib-tech-ufo-95,Lib Tech,2022,Unisex,https://www.evo.com//skis/lib-tech-ufo-95
928,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,Wood,Atomic Bent Chetler 100 Skis 2022 - Used,599.99,/used/skis/atomic-bent-chetler-100,Atomic,2022,Unisex,https://www.evo.com//used/skis/atomic-bent-che...
929,All-Mountain,No Bindings,Intermediate-Advanced,Camber/Rocker/Camber,Medium,"Basalt, Carbon, Flax, Metal, Wood",Salomon QST Lux 92 Skis - Women's - Used,499.99,/outlet/used/skis/salomon-qst-lux-92-womens,Salomon,0,Women's,https://www.evo.com//outlet/used/skis/salomon-...


In [298]:
condition= ["used", "Used"]
def find_condition_list(text, condition_list):
    for condition in condition_list:
        if condition in text:
            return "Used" 
        else:
            continue
    return "New"

In [299]:
data_df["condition"]= data_df["name"].apply(lambda x: find_condition_list(x, condition))
data_df.head()

,terrain,skis,ability_level,rocker_type,turning_radius,core_laminates,name,price,detail_link,brand,year,gender,full_link,condition
0,All-Mountain,No Bindings,Advanced-Expert,Rocker/Camber/Rocker,Medium,"Carbon, Metal, Wood",Fischer Ranger 102 FR Skis 2022,699.99,/skis/fischer-ranger-102-fr,Fischer,2022,Unisex,https://www.evo.com//skis/fischer-ranger-102-fr,New
1,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Short,Wood,Rossignol Black Ops Stargazer Skis - Women's 2022,599.95,/skis/rossignol-black-ops-stargazer-womens,Rossignol,2022,Women's,https://www.evo.com//skis/rossignol-black-ops-...,New
2,"Carving, All-Mountain",With Bindings,Beginner-Intermediate,Rocker/Camber,Short,"Composite, Wood",Head V-Shape V4 Skis ​+ PR 10 GW Bindings 2022,499.00,/ski-packages/head-v-shape-v4-skis-pr-10-gw-bi...,Head,2022,Unisex,https://www.evo.com//ski-packages/head-v-shape...,New
3,"Carving, All-Mountain",With Bindings,Intermediate-Advanced,Rocker/Camber,Short,"Graphene, Polyurethane (PU), Wood",Head V-Shape V6 Skis ​+ PR 11 GW Bindings 2022,599.00,/ski-packages/head-v-shape-v6-skis-pr-11-gw-bi...,Head,2022,Unisex,https://www.evo.com//ski-packages/head-v-shape...,New
4,"Carving, All-Mountain",With Bindings,Intermediate-Advanced,Rocker/Camber,Short,"Carbon, Wood",Head Total Joy Skis ​+ Joy 11 GW SLR Bindings ...,799.00,/ski-packages/head-total-joy-skis-11-gw-slr-bi...,Head,2022,Women's,https://www.evo.com//ski-packages/head-total-j...,New


In [300]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 931 entries, 0 to 930
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   terrain         931 non-null    object
 1   skis            931 non-null    object
 2   ability_level   931 non-null    object
 3   rocker_type     931 non-null    object
 4   turning_radius  931 non-null    object
 5   core_laminates  931 non-null    object
 6   name            931 non-null    object
 7   price           931 non-null    object
 8   detail_link     931 non-null    object
 9   brand           931 non-null    object
 10  year            931 non-null    int64 
 11  gender          931 non-null    object
 12  full_link       931 non-null    object
 13  condition       931 non-null    object
dtypes: int64(1), object(13)
memory usage: 102.0+ KB


In [301]:
data_df.price.unique()

array(['699.99', '599.95', '499.00', '599.00', '799.00', '699.95',
       '749.95', '499.95', '299.95', '749.99', '499.99', '549.00',
       '749.00', '549.99', '399.99', '959.90', '399.95', '649.99',
       '699.00', '569.95', '839.95', '1019.95', '719.95', '599.99',
       '649.95', '649.00', '849.95', '249.95', '679.00', '829.95',
       '799.95', '249.99', '229.95', '259.99', '239.99', '799.99',
       '739.95', '889.95', '869.95', '349.95', '899.95', '1299.00',
       '999.95', '779.00', '719.00', '879.95', '349.99', '859.00',
       '850.00', '899.99', '729.95', '279.95', '778.99', '967.00',
       '229.99', '199.95', '408.47', '948.00', '898.00', '839.88',
       '1149.90', '1189.89', '1719.84', '818.99', '1398.99', '1098.99',
       '209.99', '869.90', '1359.85', '1388.95', '798.95', '1298.99',
       '768.95', '1559.89', '1399.90', '1449.94', '1529.94', '718.99',
       '1029.94', '199.99', '849.00', '739.94', '1099.90', '848.95',
       '839.94', '599.90', '828.95', '759.90',

In [302]:
index_names = data_df[ data_df['price'] == '219.95\n\n - \n\n219.99' ].index
data_df.drop(index_names, inplace = True)


In [303]:
index_names = data_df[ data_df['price'] == 'Outlet:\n\n2266.92' ].index
data_df.drop(index_names, inplace = True)


In [304]:
data_df['price']= data_df['price'].astype(float)

In [305]:
data_df

,terrain,skis,ability_level,rocker_type,turning_radius,core_laminates,name,price,detail_link,brand,year,gender,full_link,condition
0,All-Mountain,No Bindings,Advanced-Expert,Rocker/Camber/Rocker,Medium,"Carbon, Metal, Wood",Fischer Ranger 102 FR Skis 2022,699.99,/skis/fischer-ranger-102-fr,Fischer,2022,Unisex,https://www.evo.com//skis/fischer-ranger-102-fr,New
1,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Short,Wood,Rossignol Black Ops Stargazer Skis - Women's 2022,599.95,/skis/rossignol-black-ops-stargazer-womens,Rossignol,2022,Women's,https://www.evo.com//skis/rossignol-black-ops-...,New
2,"Carving, All-Mountain",With Bindings,Beginner-Intermediate,Rocker/Camber,Short,"Composite, Wood",Head V-Shape V4 Skis ​+ PR 10 GW Bindings 2022,499.00,/ski-packages/head-v-shape-v4-skis-pr-10-gw-bi...,Head,2022,Unisex,https://www.evo.com//ski-packages/head-v-shape...,New
3,"Carving, All-Mountain",With Bindings,Intermediate-Advanced,Rocker/Camber,Short,"Graphene, Polyurethane (PU), Wood",Head V-Shape V6 Skis ​+ PR 11 GW Bindings 2022,599.00,/ski-packages/head-v-shape-v6-skis-pr-11-gw-bi...,Head,2022,Unisex,https://www.evo.com//ski-packages/head-v-shape...,New
4,"Carving, All-Mountain",With Bindings,Intermediate-Advanced,Rocker/Camber,Short,"Carbon, Wood",Head Total Joy Skis ​+ Joy 11 GW SLR Bindings ...,799.00,/ski-packages/head-total-joy-skis-11-gw-slr-bi...,Head,2022,Women's,https://www.evo.com//ski-packages/head-total-j...,New
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,Wood,Lib Tech UFO 105 Skis 2022,649.99,/skis/lib-tech-ufo-105,Lib Tech,2022,Unisex,https://www.evo.com//skis/lib-tech-ufo-105,New
927,"Park & Pipe, All-Mountain",No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,Wood,Lib Tech UFO 95 Skis 2022,599.99,/skis/lib-tech-ufo-95,Lib Tech,2022,Unisex,https://www.evo.com//skis/lib-tech-ufo-95,New
928,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,Wood,Atomic Bent Chetler 100 Skis 2022 - Used,599.99,/used/skis/atomic-bent-chetler-100,Atomic,2022,Unisex,https://www.evo.com//used/skis/atomic-bent-che...,Used
929,All-Mountain,No Bindings,Intermediate-Advanced,Camber/Rocker/Camber,Medium,"Basalt, Carbon, Flax, Metal, Wood",Salomon QST Lux 92 Skis - Women's - Used,499.99,/outlet/used/skis/salomon-qst-lux-92-womens,Salomon,0,Women's,https://www.evo.com//outlet/used/skis/salomon-...,Used


In [306]:
data_df.ability_level.unique()

array(['Advanced-Expert', 'Intermediate-Advanced',
       'Beginner-Intermediate', '13', '102 mm', '2 Years', '11', '100 mm',
       '98 mm', '15', '104 mm', '101 mm', '12', '14', 'Mid-Range', '16',
       'Powder'], dtype=object)

In [307]:
index_names = data_df[ data_df['ability_level'] == '102 mm' ].index
data_df.drop(index_names, inplace = True)


In [308]:
index_names = data_df[ data_df['ability_level'] == '13' ].index
data_df.drop(index_names, inplace = True)


In [309]:
index_names = data_df[ data_df['ability_level'] == '101 mm' ].index
data_df.drop(index_names, inplace = True)


In [310]:
index_names = data_df[ data_df['ability_level'] == '2 Years' ].index
data_df.drop(index_names, inplace = True)


In [311]:
index_names = data_df[ data_df['ability_level'] == '11' ].index
data_df.drop(index_names, inplace = True)


In [312]:
index_names = data_df[ data_df['ability_level'] == '15' ].index
data_df.drop(index_names, inplace = True)


In [313]:
index_names = data_df[ data_df['ability_level'] == '104 mm' ].index
data_df.drop(index_names, inplace = True)


In [314]:
index_names = data_df[ data_df['ability_level'] == '12' ].index
data_df.drop(index_names, inplace = True)


In [315]:
index_names = data_df[ data_df['ability_level'] == '16' ].index
data_df.drop(index_names, inplace = True)

In [316]:
index_names = data_df[ data_df['ability_level'] == 'Powder' ].index
data_df.drop(index_names, inplace = True)

In [317]:
index_names = data_df[ data_df['ability_level'] == '100 mm' ].index
data_df.drop(index_names, inplace = True)

In [318]:
index_names = data_df[ data_df['ability_level'] == '98 mm' ].index
data_df.drop(index_names, inplace = True)

In [319]:
index_names = data_df[ data_df['ability_level'] == '14' ].index
data_df.drop(index_names, inplace = True)

In [320]:
index_names = data_df[ data_df['ability_level'] == 'Mid-Range' ].index
data_df.drop(index_names, inplace = True)

In [321]:
data_df.turning_radius.unique()

array(['Medium', 'Short', 'Long', '13', '12', '16', '10', 'Carbon, Wood'],
      dtype=object)

In [322]:
index_names = data_df[ data_df['turning_radius'] == '12' ].index
data_df.drop(index_names, inplace = True)

In [323]:
index_names = data_df[ data_df['turning_radius'] == '13' ].index
data_df.drop(index_names, inplace = True)

In [324]:
index_names = data_df[ data_df['turning_radius'] == '16' ].index
data_df.drop(index_names, inplace = True)

In [325]:
index_names = data_df[ data_df['turning_radius'] == '10' ].index
data_df.drop(index_names, inplace = True)

In [326]:
index_names = data_df[ data_df['turning_radius'] == 'Carbon, Wood'].index
data_df.drop(index_names, inplace = True)

In [327]:
data_df

,terrain,skis,ability_level,rocker_type,turning_radius,core_laminates,name,price,detail_link,brand,year,gender,full_link,condition
0,All-Mountain,No Bindings,Advanced-Expert,Rocker/Camber/Rocker,Medium,"Carbon, Metal, Wood",Fischer Ranger 102 FR Skis 2022,699.99,/skis/fischer-ranger-102-fr,Fischer,2022,Unisex,https://www.evo.com//skis/fischer-ranger-102-fr,New
1,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Short,Wood,Rossignol Black Ops Stargazer Skis - Women's 2022,599.95,/skis/rossignol-black-ops-stargazer-womens,Rossignol,2022,Women's,https://www.evo.com//skis/rossignol-black-ops-...,New
2,"Carving, All-Mountain",With Bindings,Beginner-Intermediate,Rocker/Camber,Short,"Composite, Wood",Head V-Shape V4 Skis ​+ PR 10 GW Bindings 2022,499.00,/ski-packages/head-v-shape-v4-skis-pr-10-gw-bi...,Head,2022,Unisex,https://www.evo.com//ski-packages/head-v-shape...,New
3,"Carving, All-Mountain",With Bindings,Intermediate-Advanced,Rocker/Camber,Short,"Graphene, Polyurethane (PU), Wood",Head V-Shape V6 Skis ​+ PR 11 GW Bindings 2022,599.00,/ski-packages/head-v-shape-v6-skis-pr-11-gw-bi...,Head,2022,Unisex,https://www.evo.com//ski-packages/head-v-shape...,New
4,"Carving, All-Mountain",With Bindings,Intermediate-Advanced,Rocker/Camber,Short,"Carbon, Wood",Head Total Joy Skis ​+ Joy 11 GW SLR Bindings ...,799.00,/ski-packages/head-total-joy-skis-11-gw-slr-bi...,Head,2022,Women's,https://www.evo.com//ski-packages/head-total-j...,New
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,Wood,Lib Tech UFO 105 Skis 2022,649.99,/skis/lib-tech-ufo-105,Lib Tech,2022,Unisex,https://www.evo.com//skis/lib-tech-ufo-105,New
927,"Park & Pipe, All-Mountain",No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,Wood,Lib Tech UFO 95 Skis 2022,599.99,/skis/lib-tech-ufo-95,Lib Tech,2022,Unisex,https://www.evo.com//skis/lib-tech-ufo-95,New
928,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,Wood,Atomic Bent Chetler 100 Skis 2022 - Used,599.99,/used/skis/atomic-bent-chetler-100,Atomic,2022,Unisex,https://www.evo.com//used/skis/atomic-bent-che...,Used
929,All-Mountain,No Bindings,Intermediate-Advanced,Camber/Rocker/Camber,Medium,"Basalt, Carbon, Flax, Metal, Wood",Salomon QST Lux 92 Skis - Women's - Used,499.99,/outlet/used/skis/salomon-qst-lux-92-womens,Salomon,0,Women's,https://www.evo.com//outlet/used/skis/salomon-...,Used


In [328]:
data_df["gender"]= data_df.gender.replace({"Jr": "Kids"})


In [329]:
data_df["gender"]= data_df.gender.replace({"Junior": "Kids"})

In [330]:
data_df["gender"]= data_df.gender.replace({'Boys': "Kids"})


In [331]:
data_df["gender"]= data_df.gender.replace({'Girls': "Kids"})


In [332]:
data_df["gender"]= data_df.gender.replace({'Toddlers': "Kids"})


In [333]:
data_df["gender"]= data_df.gender.replace({'Unisex': "Adult"})


In [334]:
data_df["gender"]= data_df.gender.replace({"Women's": "Adult"})


In [335]:
data_df.gender.unique()

array(['Adult', 'Kids'], dtype=object)

In [336]:
data_df.core_laminates.unique()

array(['Carbon, Metal, Wood', 'Wood', 'Composite, Wood',
       'Graphene, Polyurethane (PU), Wood', 'Carbon, Wood', 'Metal, Wood',
       'Basalt, Carbon, Flax, Metal, Wood', 'Carbon, Flax, Metal, Wood',
       'Carbon, Flax, Wood', 'Aramid (Kevlar®), Carbon, Wood',
       'Carbon, Foam, Wood', 'Basalt, Metal, Wood',
       'Carbon, Graphene, Wood', 'Foam', 'Composite',
       'Composite, Graphene', 'Basalt, Foam, Metal',
       'Metal, Polyurethane (PU), Wood', 'Carbon, Graphene, Koroyd, Wood',
       'Basalt, Polyurethane (PU), Wood', 'Polyurethane (PU), Wood',
       'Bamboo, Carbon, Wood', 'Foam, Wood', 'Carbon, Composite',
       'Polyurethane (PU)', 'Bamboo, Carbon, Metal, Wood',
       'Bamboo, Carbon', 'Carbon', 'Basalt, Wood', 'Flax, Wood',
       'Bamboo, Wood', 'Graphene, Wood', 'Basalt, Carbon, Wood', 'Metal',
       'Composite, Metal, Wood', 'Foam, Polyurethane (PU), Wood',
       'Balsa, Carbon, Wood', 'Basalt, Carbon, Metal, Wood'], dtype=object)

In [337]:
import sweetviz
from sweetviz.sv_types import NumWithPercent, FeatureToProcess


In [338]:
my_report = sweetviz.analyze(data_df, target_feat= "price")
my_report.show_html("data.html")

                                             |      | [  0%]   00:00 -> (? left)

Report data.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [339]:
data_df

,terrain,skis,ability_level,rocker_type,turning_radius,core_laminates,name,price,detail_link,brand,year,gender,full_link,condition
0,All-Mountain,No Bindings,Advanced-Expert,Rocker/Camber/Rocker,Medium,"Carbon, Metal, Wood",Fischer Ranger 102 FR Skis 2022,699.99,/skis/fischer-ranger-102-fr,Fischer,2022,Adult,https://www.evo.com//skis/fischer-ranger-102-fr,New
1,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Short,Wood,Rossignol Black Ops Stargazer Skis - Women's 2022,599.95,/skis/rossignol-black-ops-stargazer-womens,Rossignol,2022,Adult,https://www.evo.com//skis/rossignol-black-ops-...,New
2,"Carving, All-Mountain",With Bindings,Beginner-Intermediate,Rocker/Camber,Short,"Composite, Wood",Head V-Shape V4 Skis ​+ PR 10 GW Bindings 2022,499.00,/ski-packages/head-v-shape-v4-skis-pr-10-gw-bi...,Head,2022,Adult,https://www.evo.com//ski-packages/head-v-shape...,New
3,"Carving, All-Mountain",With Bindings,Intermediate-Advanced,Rocker/Camber,Short,"Graphene, Polyurethane (PU), Wood",Head V-Shape V6 Skis ​+ PR 11 GW Bindings 2022,599.00,/ski-packages/head-v-shape-v6-skis-pr-11-gw-bi...,Head,2022,Adult,https://www.evo.com//ski-packages/head-v-shape...,New
4,"Carving, All-Mountain",With Bindings,Intermediate-Advanced,Rocker/Camber,Short,"Carbon, Wood",Head Total Joy Skis ​+ Joy 11 GW SLR Bindings ...,799.00,/ski-packages/head-total-joy-skis-11-gw-slr-bi...,Head,2022,Adult,https://www.evo.com//ski-packages/head-total-j...,New
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,Wood,Lib Tech UFO 105 Skis 2022,649.99,/skis/lib-tech-ufo-105,Lib Tech,2022,Adult,https://www.evo.com//skis/lib-tech-ufo-105,New
927,"Park & Pipe, All-Mountain",No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,Wood,Lib Tech UFO 95 Skis 2022,599.99,/skis/lib-tech-ufo-95,Lib Tech,2022,Adult,https://www.evo.com//skis/lib-tech-ufo-95,New
928,All-Mountain,No Bindings,Intermediate-Advanced,Rocker/Camber/Rocker,Medium,Wood,Atomic Bent Chetler 100 Skis 2022 - Used,599.99,/used/skis/atomic-bent-chetler-100,Atomic,2022,Adult,https://www.evo.com//used/skis/atomic-bent-che...,Used
929,All-Mountain,No Bindings,Intermediate-Advanced,Camber/Rocker/Camber,Medium,"Basalt, Carbon, Flax, Metal, Wood",Salomon QST Lux 92 Skis - Women's - Used,499.99,/outlet/used/skis/salomon-qst-lux-92-womens,Salomon,0,Adult,https://www.evo.com//outlet/used/skis/salomon-...,Used


In [340]:
data_age = (data_df.groupby(["turning_radius","ability_level","gender"], as_index=False).price.first())
data_age


,turning_radius,ability_level,gender,price
0,Long,Advanced-Expert,Adult,749.95
1,Long,Intermediate-Advanced,Adult,749.00
2,Medium,Advanced-Expert,Adult,699.99
3,Medium,Beginner-Intermediate,Adult,349.99
4,Medium,Intermediate-Advanced,Adult,699.95
5,Medium,Intermediate-Advanced,Kids,599.95
6,Short,Advanced-Expert,Adult,699.95
7,Short,Advanced-Expert,Kids,369.95
8,Short,Beginner-Intermediate,Adult,499.00
9,Short,Beginner-Intermediate,Kids,249.99


In [341]:
data_age = (data_df.groupby(["brand","turning_radius","ability_level","gender"], as_index=False).price.first())
data_age


,brand,turning_radius,ability_level,gender,price
0,Armada,Long,Advanced-Expert,Adult,899.95
1,Armada,Long,Intermediate-Advanced,Adult,799.95
2,Armada,Medium,Advanced-Expert,Adult,699.95
3,Armada,Medium,Intermediate-Advanced,Adult,499.95
4,Armada,Short,Advanced-Expert,Adult,749.95
...,...,...,...,...,...
158,WNDR Alpine,Medium,Advanced-Expert,Adult,699.00
159,WNDR Alpine,Medium,Intermediate-Advanced,Adult,699.00
160,ZAG,Long,Advanced-Expert,Adult,899.00
161,ZAG,Medium,Intermediate-Advanced,Adult,849.00


In [342]:
data_brand = (data_df.groupby(["brand","rocker_type","ability_level","gender"], as_index=False).price.first())
data_brand

,brand,rocker_type,ability_level,gender,price
0,Armada,Camber,Beginner-Intermediate,Kids,279.95
1,Armada,Camber,Intermediate-Advanced,Kids,299.95
2,Armada,Rocker/Camber,Intermediate-Advanced,Adult,408.47
3,Armada,Rocker/Camber/Rocker,Advanced-Expert,Adult,699.95
4,Armada,Rocker/Camber/Rocker,Intermediate-Advanced,Adult,499.95
...,...,...,...,...,...
154,WNDR Alpine,Rocker/Camber/Rocker,Advanced-Expert,Adult,699.00
155,WNDR Alpine,Rocker/Camber/Rocker,Intermediate-Advanced,Adult,699.00
156,ZAG,Rocker/Camber/Rocker,Advanced-Expert,Adult,899.00
157,ZAG,Rocker/Camber/Rocker,Intermediate-Advanced,Adult,849.00


In [343]:
df2 = (data_df.groupby(["brand","rocker_type","ability_level","core_laminates"], as_index=False).price.first())
df2

,brand,rocker_type,ability_level,core_laminates,price
0,Armada,Camber,Beginner-Intermediate,Composite,279.95
1,Armada,Camber,Intermediate-Advanced,Wood,299.95
2,Armada,Rocker/Camber,Intermediate-Advanced,"Aramid (Kevlar®), Carbon, Wood",499.95
3,Armada,Rocker/Camber,Intermediate-Advanced,"Carbon, Wood",408.47
4,Armada,Rocker/Camber/Rocker,Advanced-Expert,"Metal, Wood",749.95
...,...,...,...,...,...
250,WNDR Alpine,Rocker/Camber/Rocker,Advanced-Expert,"Polyurethane (PU), Wood",699.00
251,WNDR Alpine,Rocker/Camber/Rocker,Intermediate-Advanced,"Polyurethane (PU), Wood",699.00
252,ZAG,Rocker/Camber/Rocker,Advanced-Expert,Wood,899.00
253,ZAG,Rocker/Camber/Rocker,Intermediate-Advanced,"Aramid (Kevlar®), Carbon, Wood",859.00


In [344]:
df3 = (data_df.groupby(["rocker_type","ability_level","core_laminates"], as_index=False).price.first())
df3

,rocker_type,ability_level,core_laminates,price
0,Camber,Advanced-Expert,"Carbon, Metal, Wood",899.95
1,Camber,Advanced-Expert,"Metal, Wood",850.00
2,Camber,Advanced-Expert,Wood,399.95
3,Camber,Beginner-Intermediate,Composite,279.95
4,Camber,Beginner-Intermediate,"Composite, Wood",349.99
...,...,...,...,...
101,Rocker/Flat,Beginner-Intermediate,Polyurethane (PU),229.95
102,Rocker/Flat/Rocker,Advanced-Expert,"Metal, Wood",869.95
103,Rocker/Flat/Rocker,Advanced-Expert,Wood,899.95
104,Rocker/Flat/Rocker,Beginner-Intermediate,Composite,259.00


In [345]:
df3 = (data_df.groupby(["rocker_type","core_laminates", "gender"], as_index=False).price.first())
df3.head(35)

,rocker_type,core_laminates,gender,price
0,Camber,"Carbon, Metal, Wood",Adult,899.95
1,Camber,"Carbon, Wood",Adult,599.95
2,Camber,Composite,Kids,279.95
3,Camber,"Composite, Wood",Adult,349.99
4,Camber,"Composite, Wood",Kids,349.99
5,Camber,"Metal, Wood",Adult,699.95
6,Camber,Wood,Adult,399.95
7,Camber,Wood,Kids,299.95
8,Camber/Rocker/Camber,"Basalt, Carbon, Flax, Metal, Wood",Adult,499.99
9,Camber/Rocker/Camber,"Carbon, Metal, Wood",Adult,859.00
